In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import googlemaps
from geopy import distance
import folium

In [2]:
data = pd.read_csv('lista_final.csv',sep='|')

In [3]:
load_dotenv()

True

In [7]:
API_key=os.getenv('API')

In [8]:
gmaps_key = googlemaps.Client(key=API_key)

In [20]:
def get_coord(string):
    geocode_result = gmaps_key.geocode(string)
    try:
        lat = geocode_result[0]['geometry']['location']['lat']
        lon = geocode_result[0]['geometry']['location']['lng']
        return [lat,lon]
    except:
        return [None,None]

In [21]:
def get_distance(pointx, pointy):
    return distance.distance(pointx,pointy)

In [22]:
get_distance([-39, 99], [-40, 98])

Distance(140.44294774014736)

In [23]:
def rentas():
    rango_precios = {'1':'5000-10000','2':'10000-15000','3':'15000-20000','4':'20000-25000','5':'25000-30000','6':'30000-35000','7':'35000-40000','8':'40000-45000','9':'45000-50000','10':'>50000'}
    dummy = data.copy()
    
    indice_precios = rango_precios[input(f'Selecciona un rango de precios: {str(rango_precios)}')].split('-')
    
    dummy = dummy[dummy['Price(MX$)'].between(int(indice_precios[0]), int(indice_precios[1]))]
    
    indice_hab = int(input((f'Selecciona número de habitantes: ')))
    dummy = dummy[dummy['Bedrooms'] == indice_hab]

    coord_trab = input(f'Cual es tu ubicacion? : ')

    ubicacion = get_coord(coord_trab)
    dummy['Distance'] = dummy['Coordinates'].str.split(',').apply(\
                        lambda x:get_distance(x, ubicacion))
    
    dummy = dummy[dummy['Distance']<=5]
    
    dummy.drop_duplicates(subset='Address', inplace=True)
    
    return dummy, ubicacion

In [24]:
test, ubi = rentas()

Selecciona un rango de precios: {'1': '5000-10000', '2': '10000-15000', '3': '15000-20000', '4': '20000-25000', '5': '25000-30000', '6': '30000-35000', '7': '35000-40000', '8': '40000-45000', '9': '45000-50000', '10': '>50000'}1
Selecciona número de habitantes: 2
Cual es tu ubicacion? : Vicente beristain 56


In [25]:
ubi

[19.4108668, -99.1335738]

In [26]:
test.head()

,Address,Price(MX$),Bedrooms,Bathrooms,Size(m2),Latitud,Longitud,Coordinates,Distance
39,"Centro (área 1), Cuauhtémoc, Distrito Federa...",8500,2,1.0,72,19.430268,-99.137314,"19.430267800000003,-99.1373136",2.183255959808334 km
143,"Sur 109, Iztacalco, Distrito Federal (cdmx)",10000,2,1.0,100,19.379658,-99.127942,"19.379658300000003,-99.12794240000001",3.504958458211645 km
166,"Cumbres De Acultzingo 67, 108, Narvarte, Ben...",9500,2,1.0,70,19.392331,-99.147842,"19.3923314,-99.1478421",2.540870656758361 km
350,"Víctor Hugo 17, 401, Portales Oriente, Benit...",10000,2,1.0,80,19.368822,-99.138145,"19.3688216,-99.1381447",4.678968977910246 km
561,"Plutarco Elias Calles 282, Iztacalco, Distri...",10000,2,1.0,72,19.397825,-99.111950,"19.3978246,-99.1119502",2.69125921854357 km


depas = rentas()

depas

data['Distance'] = data['Coordinates'].apply(lambda x: get_distance(x.split(",")))

data.sort_values(by='Distance').head()

mapa = folium.Map(
    location=ubi,
    zoom_start=15,
    tiles='openstreetmap')
tooltip = 'Click me!'

for index, row in data.head(10).iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  radius=500,
                  popup=row['Address'],
                  icon=folium.Icon(color='purple',icon='home')
                 ).add_to(mapa)

mapa

In [27]:
test.drop_duplicates(subset='Address', inplace=True)

In [28]:
test.sort_values(by='Distance').head(10)

,Address,Price(MX$),Bedrooms,Bathrooms,Size(m2),Latitud,Longitud,Coordinates,Distance
39,"Centro (área 1), Cuauhtémoc, Distrito Federa...",8500,2,1.0,72,19.430268,-99.137314,"19.430267800000003,-99.1373136",2.183255959808334 km
166,"Cumbres De Acultzingo 67, 108, Narvarte, Ben...",9500,2,1.0,70,19.392331,-99.147842,"19.3923314,-99.1478421",2.540870656758361 km
561,"Plutarco Elias Calles 282, Iztacalco, Distri...",10000,2,1.0,72,19.397825,-99.111950,"19.3978246,-99.1119502",2.69125921854357 km
143,"Sur 109, Iztacalco, Distrito Federal (cdmx)",10000,2,1.0,100,19.379658,-99.127942,"19.379658300000003,-99.12794240000001",3.504958458211645 km
581,"Sur 143 1614, Iztacalco, Distrito Federal (c...",7000,2,1.0,120,19.387874,-99.098426,"19.3878741,-99.09842579999999",4.4842124292550505 km
350,"Víctor Hugo 17, 401, Portales Oriente, Benit...",10000,2,1.0,80,19.368822,-99.138145,"19.3688216,-99.1381447",4.678968977910246 km


In [29]:
mapa2 = folium.Map(
    location=ubi,
    zoom_start=12,
    tiles='openstreetmap')
tooltip = 'Click me!'


for index, row in test.sort_values(by='Distance').head(10).iterrows():
    folium.Marker([row['Latitud'], row['Longitud']], 
                  radius=500,
                  popup=row[['Address','Bedrooms','Price(MX$)']],
                  icon=folium.Icon(color='purple',icon='home')
                 ).add_to(mapa2)
    
folium.Marker(ubi, icon=folium.Icon(color='blue',icon='home')).add_to(mapa2)

mapa2